# What is Selenium?

## 1

What environment do we use?

See [What is Docker?](What%20is%20Docker.ipynb)

Run Selenium in Docker?


In [5]:
var importer = require('../Core');
var path = require('path');
var writeFileSync = require('fs').writeFileSync;

// add some run commands to the bash script
var bashToRun = (code) => code.split('\n').filter(n => n.trim().length > 0).map(l => 'RUN ' + l.trim())
    .join('\n').replace(/\\\s*\nRUN\s*/ig, '\\\n ');

// create a selenium Dockerfile with a vnc connection
var seleniumDocker = (outputFile) => {
    return importer.interpretAll([
        'run selenium',
        // add some extra services
        'linux dev tools',
        'vnc html',
        'vnc docker'
    ]).then(r => {
        // convert some results to Docker RUN commands instead of bash
        r[1].code = bashToRun(r[1].code);
        r[2].code = bashToRun(r[2].code);
        return r.map(r => r.code).join('\n');
    })
    .then(r => {
        // save the Dockerfile
        writeFileSync(outputFile, r);
        return r;
    });
};
(seleniumDocker);


[Function: seleniumDocker]

How to set up selenium server?

In [14]:
var importer = require('../Core');
var path = require('path');

var DOWNLOAD_DIR = path.join(process.env.HOME || process.env.HOMEPATH || process.env.USERPROFILE, 'Downloads');
var PROFILE_DIR = path.join(process.env.HOME || process.env.HOMEPATH || process.env.USERPROFILE, '.defaultProfile');

var execCmd;
var getSeleniumServer = (name = 'act-selenium') => {
    try {
        fs.mkdirSync(DOWNLOAD_DIR);
    } catch (err) {
        if (err.code != 'EEXIST') { throw err; }
    }
    try {
        fs.mkdirSync(PROFILE_DIR);
    } catch (err) {
        if (err.code != 'EEXIST') { throw err; }
    }
    try {
        fs.unlinkSync(path.join(PROFILE_DIR, 'SingletonLock'));
    } catch (err) {
        if (err.code != 'ENOENT') { throw err; }
    }
    var DOCKERFILE = path.resolve(path.join(__dirname, 'Dockerfile'));
    return seleniumDocker(DOCKERFILE)
        .then(() => importer.interpretAll('spawn child process'))
        .then(r => {
        console.log(r);
            execCmd = eval("'use strict';" + r[0].code);
            return execCmd('docker ps -a');
        }).then(r => {
            var build = 'docker build -t ' + name + ' "' 
                + path.dirname(DOCKERFILE) + '"\n'
                + (r[0].indexOf(name) > -1 
                   ? ('docker stop ' + name + '\n') : '')
                + 'docker run --rm --shm-size=2g -d '
                + '--name ' + name + ' ' 
                + '-p 8888:8888 '
                + '-p 6080:6080 '
                + '-p 5900:5900 '
                + '-p 4444:4444 '
                + '-p 4200:4200 '
                + '-p 3000:3000 '
                // TODO: add profile dir back in when permissions works on windows
                + '-v "' + DOWNLOAD_DIR + '":/data/downloads ' 
                + name + '\n';
            return execCmd(build)
        }).then(r => new Promise(resolve => setTimeout(() => resolve(r), 5000)));
};
(getSeleniumServer);

[Function: getSeleniumServer]

In [15]:
$$.async();
try {
getSeleniumServer()
    .then(r => $$.sendResult(r))
    .catch(e => $$.sendError(e));
} catch (e) {
    $$.sendError(e);
}


Already imported
[ 'import notebook.ipynb', 'interpret all notebooks.ipynb' ]
Already imported
[ 'import notebook.ipynb', 'interpret all notebooks.ipynb' ]
[ { code: 'var exec = require(\'child_process\').exec;\n// read cmd template and execute each line?\nvar corePath = path.resolve(path.join(__dirname, \'..\', \'Core\'));\nvar importer = require(corePath);\n\nvar execCmd = (script, vars = {}, options = {}) => {\n    // add some run commands to the bash script\n    var bashToExec = (code) => code.split(/\\s*(\\n\\s*)+/ig).map(l => \'{EXEC} \' + l)\n        .join(\'\\n\').replace(/\\\\\\s*\\n\\{EXEC\\}\\s*/ig, \'\\\\\\n \');\n    \n    return importer.runAllPromises(\n        bashToExec(script).split(/\\s*\\{EXEC\\}\\s*/ig)\n        .filter(r => r.trim() !== \'\')\n        .map(r => new Promise((resolve, reject) => {\n            exec(r, (err, result) => {\n                if(err) {\n                    return resolve(err.toString());\n                }\n                resolve(result.

[ 'Sending build context to Docker daemon  77.31kB\r\r\nStep 1/22 : FROM selenium/standalone-chrome-debug\n ---> 593e47f379db\nStep 2/22 : EXPOSE 4200\n ---> Using cache\n ---> cf96a9aac1f7\nStep 3/22 : EXPOSE 4444\n ---> Using cache\n ---> 3945a29b8f5b\nStep 4/22 : EXPOSE 3000\n ---> Using cache\n ---> b8b75fe784a3\nStep 5/22 : ENV CHROME_USER_DATA_DIR /usr/profile\n ---> Using cache\n ---> 00409baf3489\nStep 6/22 : WORKDIR /home/seluser\n ---> Using cache\n ---> b8a12d57779f\nStep 7/22 : USER root\n ---> Using cache\n ---> aaa0d8320a97\nStep 8/22 : RUN mkdir /usr/profile\n ---> Using cache\n ---> 62eef137b4b0\nStep 9/22 : RUN mkdir /usr/downloads\n ---> Using cache\n ---> bf11798fa872\nStep 10/22 : RUN chown seluser:seluser -R /usr/profile\n ---> Using cache\n ---> 6cd96f04f28a\nStep 11/22 : RUN chown seluser:seluser -R /usr/downloads\n ---> Using cache\n ---> 52b7a641fd74\nStep 12/22 : RUN chmod 777 -R /usr/profile\n ---> Using cache\n ---> 0b526765e8e3\nStep 13/22 : RUN chmod 777 -

# 2

How to connect to VNC through the web browser?

[Click here to open](http://localhost:6080/vnc.html?password=secret&host=localhost&port=6080&autoconnect=true&resize=downscale&view_only=true&reconnect=true) in a separate browser


In [5]:
var vncIframe = () => `
<div style="display:block; width:100%; padding-bottom:70%;position:relative;">
<iframe id="vnc"
style="position:absolute;top:0;right:0;bottom:0;left:0;width:100%;height:100%;border:0;" 
src="//local:6080/vnc.html?password=secret&host=localhost&port=6080&autoconnect=true&resize=downscale&view_only=true&reconnect=true"></iframe></div>
`;
(vncIframe);

[Function: vncIframe]

In [6]:
$$.mime({'text/html': vncIframe()});


## 3

How to run a selenium test on the Docker machine?

Auto fill fields in a live browser using the 

Transfer login state to current browser?



In [24]:
console.log('hit');
client.url('https://purchasesprint.actops.com');
client.click('[href*="/auth"], [routerlink*="/auth"]');


module.exports = client;

hit


{ state: 'fulfilled', value: undefined }

In [2]:
var importer = require('../Core');

$$.async();
var client;
importer.interpretAll(['webdriver client', 'run a selenium test'])
    .then(r => r[0].runInNewContext())
    .then(r => client = r())
    .then(() => importer.interpretAll(['run a selenium test']))
    .then(r => r[0].runFresherInNewContext({client: client}))
    .then(r => $$.sendResult(r))
    .catch(e => $$.sendError(e));



Already imported
[ 'import notebook.ipynb', 'interpret all notebooks.ipynb' ]
Initializing webdriver on localhost
Already imported
[ 'import notebook.ipynb', 'interpret all notebooks.ipynb' ]
hit
